In [8]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_only import*
tox_param_file = os.path.join(root_path,'iltransr/pre-trained params/TOX_best.params')

In [9]:
cnn_dropout = 0.1
batch_size = 64
bucket_num, bucket_ratio = 2, 0

In [ ]:
net = ILNet(dropout=cnn_dropout)
net.encoder = model.encoder
net.src_embed =  model.src_embed
net.hybridize()

In [11]:
net.load_parameters(tox_param_file,ctx=ctx)
net.initialize(init=mx.init.Xavier(magnitude=magnitude), ctx=ctx)

In [14]:
import pandas as pd
to_predict_data = pd.read_excel('smiles.xlsx',sheet_name='To_be_calculated')

In [15]:
predict_smiles = to_predict_data['SMILES'].map(canonical_smile).map(no_split).map(preprocess)
predict_smiles_lengths = to_predict_data['SMILES'].map(canonical_smile).map(no_split).map(get_length)
predict_sigma =mx.nd.ones((len(predict_smiles),1))
predict_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(predict_smiles,predict_sigma))
predict_dataloader = get_predict_dataloader(predict_dataset,predict_smiles_lengths,batch_size=batch_size, bucket_num=bucket_num,bucket_ratio=bucket_ratio)

In [16]:
predicted = predict(net, predict_dataloader,ctx)

[02:42:25] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[02:42:26] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


In [17]:
df = pd.DataFrame(predicted)
df.to_csv('predicted_TOX.csv')
print(df)

             0
0     3.161665
1     3.351935
2     0.661414
3     0.433585
4     3.007277
...        ...
5513  3.854147
5514  3.805838
5515  0.603192
5516  3.329491
5517  3.014625

[5518 rows x 1 columns]
